## A notebook to collate data from Dimensions Google BigQuery database

### Last updated: 19th January 2023

#### Authors: Charlie Rahal and Saurabh Khanna

Load dependancies:

In [1]:
import pandas as pd
import os
from google.cloud import bigquery
from helper_functions import get_pubs_all_issn

Authenticate with to the bigquery client and project:

In [2]:
MY_PROJECT_ID = "dimensionspkp"
client = bigquery.Client(project=MY_PROJECT_ID)

Load the raw list of issns

In [3]:
raw_path = os.path.join("..", "data", "raw")
raw_data = pd.read_csv(os.path.join(raw_path, "ojs_issns_in_dimensions_2020.csv"))
issns_to_query = raw_data["issn"].tolist()
print("Total ISSNs to query: ", len(issns_to_query))

Total ISSNs to query:  12435


Set a raw dimensions output path:

In [ ]:
dim_out = os.path.join('..', 'data', 'raw', 'from_dimensions')

Call our helper function to grab the data for all pubs in our ISSN list. Note: we chunk into 1000 issns to get over local memory file limits.

*Question*: Is each call of `pub_from_issn_lists` costing GBQ $, or is passing a list as `(%s)""" %(issn_list)` costing credits _for each element of the list_? How else can we return all pub.ids from our ISSN list?

In [4]:
%%time
file_name = 'pubs_from_all_issns.csv'
file_path = os.path.join(dim_out, file_name)
get_pubs_all_issn(1000, file_path, client, issns_to_query)

13it [1:25:53, 396.42s/it]

CPU times: user 24min 18s, sys: 30.8 s, total: 24min 49s
Wall time: 1h 25min 53s


Lets now get a list of all unique publications found in our raw data returned from the above function calls

In [32]:
unique_pubids = pd.read_csv(file_path, usecols=[1], header=None)
unique_pubids = unique_pubids[1].tolist()
print('# pubids from issns: ', len(unique_pubids))
# Note: should be the same as the set:
print('# unique pubids from issns: ', len(set(unique_pubids)))

# pubids from issns:  1947263
# unique pubids from issns:  1947263


Let's now work on using this list of pubids to get incoming citations:

In [137]:
%%time
# Prototype the fetch_level1_incoming functions

def fetch_level1_incoming(issn_list):
    """
    A function to grab level1 incoming citations.
    :param issn_list: a list of issns (not implemented)
    :return: rows of data
    """
    try:
        QUERY = """
                SELECT id, title.preferred, doi, journal.issn,
                       publisher, type, date, open_access_categories,
                       category_for, citations_count, journal,
                       researcher_ids, research_orgs, research_org_cities,
                       research_org_city_names, research_org_countries,
                       research_org_country_names, concepts,
                       reference_ids
                FROM
                  `dimensions-ai.data_analytics.publications`
                WHERE
                  id IN (
                  SELECT
                    DISTINCT reference_pubs
                  FROM
                    `dimensions-ai.data_analytics.publications`,
                    UNNEST(reference_ids) AS reference_pubs
                  WHERE
                  --- "jour.1115214" is a test from the docs
                    journal.id = "jour.1115214")
                """

        query_job = client.query(QUERY)  # API request
        rows = query_job.result()  # Wait for query to finish
        return rows
    except Exception as e:
        print(e.message)

# Question: how can we avoid calling this 12435 times?
# That seems to be very expensive via GBQ?
results = fetch_level1_incoming(unique_pubids[0:1])

CPU times: user 17.5 ms, sys: 3.85 ms, total: 21.4 ms
Wall time: 14.5 s


In [133]:
%%time
# Question: how can we avoid calling this so many times?
results.to_dataframe().to_csv('test.csv')